**Name: Daksh.M.Dholakia**

**SAPID: 60009200040**

**Batch: D11**

**Subject: ADS Laboratory**

# **Lab 7: Modelling and Optimisation Trade-off**

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import pickle as pc
import pandas as pd
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV

In [ ]:
data = pd.read_csv('train.csv')
print(data.shape)

(1460, 81)


In [ ]:
missing_data = data.isnull().sum()
print("Missing Data:")
print(missing_data[missing_data > 0])

Missing Data:
LotFrontage      259
Alley           1369
MasVnrType         8
MasVnrArea         8
BsmtQual          37
BsmtCond          37
BsmtExposure      38
BsmtFinType1      37
BsmtFinType2      38
Electrical         1
FireplaceQu      690
GarageType        81
GarageYrBlt       81
GarageFinish      81
GarageQual        81
GarageCond        81
PoolQC          1453
Fence           1179
MiscFeature     1406
dtype: int64


In [ ]:
columns_to_drop = ['PoolQC', 'Fence', 'MiscFeature', 'Alley', 'FireplaceQu']
data = data.drop(columns=columns_to_drop)

In [ ]:
numeric_columns_with_missing = ['MasVnrArea', 'GarageYrBlt','LotFrontage']
data[numeric_columns_with_missing] = data[numeric_columns_with_missing].fillna(data[numeric_columns_with_missing].mean())
categorical_columns_with_missing = ['MasVnrType', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Electrical', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond']
data[categorical_columns_with_missing] = data[categorical_columns_with_missing].fillna('NA')
missing_data_after_handling = data.isnull().sum()
print("\n \nMissing Data After Handling:")
print(missing_data_after_handling[missing_data_after_handling > 0])


 
Missing Data After Handling:
Series([], dtype: int64)


In [ ]:
data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,...,0,0,0,0,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,...,0,0,0,0,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,...,0,0,0,0,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,...,272,0,0,0,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,...,0,0,0,0,0,12,2008,WD,Normal,250000


In [ ]:
columnsdrop=['Id','LotShape','MSZoning', 'Street', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'SaleType', 'SaleCondition']
data = data.drop(columns=columnsdrop)

In [ ]:
X = data.drop('SalePrice', axis=1)
y = data['SalePrice']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
data.columns

Index(['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond',
       'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2',
       'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces',
       'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF',
       'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal',
       'MoSold', 'YrSold', 'SalePrice'],
      dtype='object')

In [ ]:
data.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
0,60,65.0,8450,7,5,2003,2003,196.0,706,0,...,0,61,0,0,0,0,0,2,2008,208500
1,20,80.0,9600,6,8,1976,1976,0.0,978,0,...,298,0,0,0,0,0,0,5,2007,181500
2,60,68.0,11250,7,5,2001,2002,162.0,486,0,...,0,42,0,0,0,0,0,9,2008,223500
3,70,60.0,9550,7,5,1915,1970,0.0,216,0,...,0,35,272,0,0,0,0,2,2006,140000
4,60,84.0,14260,8,5,2000,2000,350.0,655,0,...,192,84,0,0,0,0,0,12,2008,250000


In [ ]:
ridge_model = Ridge(alpha = 0.1, solver ='saga')
ridge_model.fit(X_train, y_train)
y_pred_ridge = ridge_model.predict(X_test)
mse_ridge = mean_squared_error(y_test, y_pred_ridge)
mae_ridge = mean_absolute_error(y_test, y_pred_ridge)
r2_ridge = r2_score(y_test, y_pred_ridge)
print(f"Mean Squared Error (MSE): {mse_ridge}")
print(f"Mean Absolute Error (MAE): {mae_ridge}")
print(f"R-squared (R^2): {r2_ridge}")

Mean Squared Error (MSE): 1964478141.9693997
Mean Absolute Error (MAE): 28163.39852882241
R-squared (R^2): 0.7438859718731812


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [ ]:
alphas = [0.001, 0.01, 0.1, 1, 10, 100]
solvers = ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga']
param_grid = {'alpha': alphas, 'solver': solvers}
ridge_grid = GridSearchCV(Ridge(), param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

In [ ]:
ridge_grid.fit(X_train, y_train)
best_alpha = ridge_grid.best_params_['alpha']
best_solver = ridge_grid.best_params_['solver']
best_ridge_model = Ridge(alpha=best_alpha, solver=best_solver)
best_ridge_model.fit(X_train, y_train)
y_pred_ridge = best_ridge_model.predict(X_test)
mse_ridge = mean_squared_error(y_test, y_pred_ridge)
mae_ridge = mean_absolute_error(y_test, y_pred_ridge)
r2_ridge = r2_score(y_test, y_pred_ridge)

In [ ]:
print(f"Best Alpha: {best_alpha}")
print(f"Best Solver: {best_solver}")
print(f"Mean Squared Error (MSE): {mse_ridge}")
print(f"Mean Absolute Error (MAE): {mae_ridge}")
print(f"R-squared (R^2): {r2_ridge}")

Best Alpha: 10
Best Solver: auto
Mean Squared Error (MSE): 1355827134.1639595
Mean Absolute Error (MAE): 22950.322408673546
R-squared (R^2): 0.823237356855365


In [ ]:
with open('ridge_model.pkl', 'wb') as model_file:
    pc.dump(best_ridge_model, model_file)

In [ ]:
!jupyter nbconvert --to html "/content/60009200040_ADS_D11_Lab7.ipynb"

[NbConvertApp] Converting notebook /content/60009200040_ADS_D11_Lab7.ipynb to html
[NbConvertApp] Writing 625088 bytes to /content/60009200040_ADS_D11_Lab7.html
